In [1]:
import pandas as pd
import re
import json
import os
import numpy as np
from tqdm import tqdm

In [2]:
absolute_path = '/data/UCSD_cxr/'
through_2020_df = pd.read_csv(absolute_path + 'all-reports-available-xrays-through2020.csv')
through_2020_df.dropna(inplace=True)
after_2020_df = pd.read_csv(absolute_path + 'all-reports-available-xrays-after2020.csv')
after_2020_df.dropna(inplace=True)

/tmp/ipykernel_743831/1910024422.py:2: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  through_2020_df = pd.read_csv(absolute_path + 'all-reports-available-xrays-through2020.csv')


In [3]:
def adjust_report(row):
    if row['LastTitle'] != '':
        return row['ReportText']
    else:
        return row['ReportText'] + '\nEND:\n'

def extract_to_next_title(text, start_title):
    # Regular expression pattern: start_title followed by any characters until the next all-capitalized title
    pattern = rf"{start_title}:\s*(.*?)(?=\b[A-Z\s]+:)"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else None

def is_impression_last_title(text):
    # Regular expression to find all capitalized titles followed by a colon
    titles = re.findall(r"\b[A-Z\s]+:\s*", text)
    cleaned_titles = [t.strip() for t in titles]
    end_with_impression = 'IMPRESSION:' in cleaned_titles and cleaned_titles[-1] == 'IMPRESSION:'
    # Check if 'IMPRESSION' is in the list and if it's the last element
    if len(cleaned_titles) == 0:
        return 'invalid'
    return '' if end_with_impression else cleaned_titles[-1]

In [4]:
through_2020_df['LastTitle'] = through_2020_df['ReportText'].apply(is_impression_last_title)
after_2020_df['LastTitle'] = after_2020_df['ReportText'].apply(is_impression_last_title)
through_2020_df = through_2020_df[through_2020_df['LastTitle'] != 'invalid']
after_2020_df = after_2020_df[after_2020_df['LastTitle'] != 'invalid']

through_2020_df['AdjustedReportText'] = through_2020_df.apply(adjust_report, axis = 1)
after_2020_df['AdjustedReportText'] = after_2020_df.apply(adjust_report, axis = 1)
after_2020_df['AccessionId'] = after_2020_df['AccessionId'].map(str)

through_2020_df.head()

,AccessionId,DocumentTitle,DocumentDate,Author,ReportText,LastTitle,AdjustedReportText
0,44067089,X-RAY CHEST SINGLE VIEW,11/1/2017,"Kligerman, Seth",EXAM DESCRIPTION: *\n\nCHEST X-RAY 1 VW(7101...,CONCURRENT SUPERVISION:,EXAM DESCRIPTION: *\n\nCHEST X-RAY 1 VW(7101...
1,44067059,X-RAY CHEST SINGLE VIEW,11/1/2017,"Kligerman, Seth",EXAM DESCRIPTION: *\n\nCHEST X-RAY 1 VW(7101...,CONCURRENT SUPERVISION:,EXAM DESCRIPTION: *\n\nCHEST X-RAY 1 VW(7101...
2,44066941,X-RAY CHEST SINGLE VIEW,11/1/2017,"Ladd, William",EXAM DESCRIPTION: *\n\nCHEST X-RAY 1 VW(7101...,CONCURRENT SUPERVISION:,EXAM DESCRIPTION: *\n\nCHEST X-RAY 1 VW(7101...
3,44066704,X-RAY CHEST SINGLE VIEW,11/1/2017,"Naheedy, John",EXAM DESCRIPTION: *\n\nCHEST X-RAY 1 VW(7101...,,EXAM DESCRIPTION: *\n\nCHEST X-RAY 1 VW(7101...
4,44066777,X-RAY CHEST SINGLE VIEW,11/1/2017,"Kligerman, Seth",EXAM DESCRIPTION: *\n\nCHEST X-RAY 1 VW(7101...,CONCURRENT SUPERVISION:,EXAM DESCRIPTION: *\n\nCHEST X-RAY 1 VW(7101...


In [5]:
after_2020_df.head()

,FalseID,AccessionId,DocumentTitle,DocumentDate,Context,Age,Sex,Author,ReportText,LastTitle,AdjustedReportText
0,1567456,53333541,X-RAY CHEST SINGLE VIEW,1/23/2021,Outpatient,52,Male,"Jacobs, Kathleen",EXAM DESCRIPTION: X-RAY CHEST SINGLE VIEW ...,,EXAM DESCRIPTION: X-RAY CHEST SINGLE VIEW ...
1,28204320,53205565,X-RAY CHEST SINGLE VIEW,12/17/2020,Inpatient,79,Female,"Kligerman, Seth",EXAM DESCRIPTION: X-RAY CHEST SINGLE VIEW ...,,EXAM DESCRIPTION: X-RAY CHEST SINGLE VIEW ...
2,28240024,53993975,X-RAY CHEST FRONTAL AND LATERAL,9/2/2021,Inpatient,70,Male,"Hahn, Lewis",EXAM DESCRIPTION: X-RAY CHEST FRONTAL AND LAT...,,EXAM DESCRIPTION: X-RAY CHEST FRONTAL AND LAT...
3,28254880,53513416,X-RAY CHEST SINGLE VIEW,3/28/2021,Emergency,62,Female,"Hahn, Lewis",EXAM DESCRIPTION: X-RAY CHEST SINGLE VIEW ...,,EXAM DESCRIPTION: X-RAY CHEST SINGLE VIEW ...
4,28260488,53662146,X-RAY CHEST SINGLE VIEW,5/17/2021,Outpatient,81,Female,"Hahn, Lewis",EXAM DESCRIPTION: X-RAY CHEST SINGLE VIEW ...,,EXAM DESCRIPTION: X-RAY CHEST SINGLE VIEW ...


In [6]:
print(through_2020_df.loc[3, 'AdjustedReportText'])

EXAM DESCRIPTION:   *

CHEST X-RAY 1 VW(71010) Qty:1

CLINICAL HISTORY:   
The increasing CO2

COMPARISON:   
Previous day

FINDINGS:   
See Impression

IMPRESSION:   
ET tube tip is at the thoracic inlet. Ductal clip, enteric tube, and right upper extremity PICC line are unchanged.


The lungs are less well expanded with increased perihilar and left basilar atelectasis. This superimposed on evolving chronic lung changes throughout both lungs. No large effusions, or pneumothoraces are noted. The cardiac silhouette is unchanged.





END:



In [7]:
print(extract_to_next_title(through_2020_df.loc[3, 'AdjustedReportText'], 'CLINICAL HISTORY'))
print(extract_to_next_title(through_2020_df.loc[3, 'AdjustedReportText'], 'FINDINGS'))
print(extract_to_next_title(through_2020_df.loc[3, 'AdjustedReportText'], 'IMPRESSION'))

The increasing CO2
See Impression
ET tube tip is at the thoracic inlet. Ductal clip, enteric tube, and right upper extremity PICC line are unchanged.


The lungs are less well expanded with increased perihilar and left basilar atelectasis. This superimposed on evolving chronic lung changes throughout both lungs. No large effusions, or pneumothoraces are noted. The cardiac silhouette is unchanged.


In [8]:
print(through_2020_df.loc[0, 'AdjustedReportText'])

EXAM DESCRIPTION:   *

CHEST X-RAY 1 VW(71010) Qty:1

CLINICAL HISTORY:   
Concern for aspiration

TECHNIQUE:   
AP chest radiograph

COMPARISON:   *

10/31/2017

FINDINGS:   

*

Lines and Tubes: None


Mediastinum: The cardiomediastinal silhouette is unchanged. No lymphadenopathy is appreciated.

*

Lungs: The lungs are clear.

*

Pleura: No pneumothorax or effusion.

*

Bones and soft tissues: Unchanged

IMPRESSION:   
Clear lungs
*


CONCURRENT SUPERVISION:







In [9]:
print(extract_to_next_title(through_2020_df.loc[0, 'AdjustedReportText'], 'CLINICAL HISTORY'))
print(extract_to_next_title(through_2020_df.loc[0, 'AdjustedReportText'], 'FINDINGS'))
print(extract_to_next_title(through_2020_df.loc[0, 'AdjustedReportText'], 'IMPRESSION'))

Concern for aspiration
*

Lines and Tubes: None


Mediastinum: The cardiomediastinal silhouette is unchanged. No lymphadenopathy is appreciated.

*

Lungs: The lungs are clear.

*

Pleura: No pneumothorax or effusion.

*

Bones and soft tissues: Unchanged
Clear lungs
*


In [10]:
print(after_2020_df.loc[1, 'AdjustedReportText'])

EXAM DESCRIPTION:  X-RAY CHEST SINGLE VIEW    CLINICAL HISTORY:  Neck and sternal pain    COMPARISON:  11/28/2020    FINDINGS:    Lines and Tubes: None  Mediastinum: The cardiomediastinal silhouette is unchanged. Atherosclerotic disease  Lungs: Emphysema  Pleura: No pneumothorax or effusion.  Bones and soft tissues: Unchanged             Signed by: Kligerman, Seth 12/17/2020 16:46:34  IMPRESSION:  IMPRESSION:  Smoking related lung disease. No evidence of pneumonia or explanation for pain  
END:



In [11]:
print(extract_to_next_title(after_2020_df.loc[1, 'AdjustedReportText'], 'CLINICAL HISTORY'))
print(extract_to_next_title(after_2020_df.loc[1, 'AdjustedReportText'], 'FINDINGS'))
print(extract_to_next_title(after_2020_df.loc[1, 'AdjustedReportText'], 'IMPRESSION:  IMPRESSION'))

Neck and sternal pain
Lines and Tubes: None  Mediastinum: The cardiomediastinal silhouette is unchanged. Atherosclerotic disease  Lungs: Emphysema  Pleura: No pneumothorax or effusion.  Bones and soft tissues: Unchanged             Signed by: Kligerman, Seth 12/17/2020 16:46:34
Smoking related lung disease. No evidence of pneumonia or explanation for pain


In [12]:
all_jpgs = list(os.listdir('/data/UCSD_cxr/jpg'))
all_jpgs_dict = {jpg.split('_')[1][:-4] : jpg for jpg in all_jpgs}
all_jpgs_dict

{'50523193': 'Rosceelkel_50523193.jpg',
 '54256937': 'Quashiha_54256937.jpg',
 '52158712': 'Dupudo_52158712.jpg',
 '53371690': 'Queprifri_53371690.jpg',
 '53601492': 'Notehaf_53601492.jpg',
 '52470220': 'Numeypil_52470220.jpg',
 '54167086': 'Swuhimo_54167086.jpg',
 '53655193': 'Alnafep_53655193.jpg',
 '52486832': 'Catofil_52486832.jpg',
 '53970841': 'Bireskas_53970841.jpg',
 '53540538': 'Vudece_53540538.jpg',
 '52755781': 'Clagemo_52755781.jpg',
 '53658238': 'Lojise_53658238.jpg',
 '51951596': 'Noparot_51951596.jpg',
 '53237729': 'Foquasna_53237729.jpg',
 '53581568': 'Quenuba_53581568.jpg',
 '53864880': 'Bebukom_53864880.jpg',
 '50888653': 'Ceijechet_50888653.jpg',
 '53858868': 'Gumisen_53858868.jpg',
 '54316439': 'Taquegi_54316439.jpg',
 '51533368': 'Prierune_51533368.jpg',
 '53495210': 'Onkurit_53495210.jpg',
 '53864344': 'Polulep_53864344.jpg',
 '52567685': 'Inhenus_52567685.jpg',
 '53632472': 'Apiduf_53632472.jpg',
 '52520842': 'Okarol_52520842.jpg',
 '53379482': 'Dunobom_53379482.

In [13]:
test_df = pd.read_csv('/data/UCSD_cxr/test_accn.csv')
train_df = pd.read_csv('/data/UCSD_cxr/train_accn.csv')
test_df.head(10)

,Unnamed: 0,FalseID,Accession
0,93,28369992,53321530
1,96,28422472,50388616
2,98,28447808,54129354
3,99,28447808,50001322
4,100,28447808,52403905
5,101,28447808,52295716
6,102,28447808,52444830
7,103,28447808,53473274
8,104,28447808,53477011
9,105,28447808,53472272


In [14]:
test_set_accn = set(test_df['Accession'].astype(str))
actual_present = set(all_jpgs_dict.keys())
print(len(test_set_accn.intersection(actual_present)))

2377


In [15]:
train_set_accn = set(train_df['Accession'].astype(str))
actual_present_train = set(all_jpgs_dict.keys())
print(len(train_set_accn.intersection(actual_present_train)))

110452


In [16]:
json_file = []
jsonl_file = []
save_json_fp = '/home/llm-hackathon/LLaVA/muchan/patient_finding_impression.json'
qa_jsonl_fp = '/home/llm-hackathon/LLaVA/muchan/testset_question_finding_impression.jsonl'
question = '\nBased on AUTHOR and CLINICAL HISTORY information, suppose you were the named specialist AUTHOR, provide me a detailed report reading this chest X-ray as closely resembling the style of this AUTHOR as possible.'
through_2020_df = through_2020_df.drop_duplicates()
after_2020_df = after_2020_df.drop_duplicates()
through_2020_df.reset_index(drop=True, inplace=True)
after_2020_df.reset_index(drop=True, inplace=True)

through_2020_count = 0
after_2020_count = 0
total_test_count = 0
for i in tqdm(range(through_2020_df.shape[0])):
    accession_id = str(through_2020_df.loc[i, 'AccessionId'])
    author = 'AUTHOR: ' + through_2020_df.loc[i, 'Author']
    try:
        clinical_history = 'CLINICAL HISTORY: ' + extract_to_next_title(through_2020_df.loc[i, 'AdjustedReportText'], 'CLINICAL HISTORY')
    except:
        clinical_history = 'CLINICAL HISTORY: None'
    prompt = author + clinical_history + question
    if accession_id in all_jpgs_dict and accession_id in train_set_accn:
        try:
            findings = 'FINDINGS: ' + extract_to_next_title(through_2020_df.loc[i, 'AdjustedReportText'], 'FINDINGS')
        except:
            findings = 'FINDINGS: See IMPRESSION'
        try:
            impression = 'IMPRESSION: ' + extract_to_next_title(through_2020_df.loc[i, 'AdjustedReportText'], 'IMPRESSION')
        except:
            continue
        answer = findings + impression
        patient_data = {
            "id": all_jpgs_dict[accession_id],
            "image": '/data/UCSD_cxr/jpg/' + all_jpgs_dict[accession_id],
            "conversations": [
                {
                    "from": "human",
                    "value": f"{prompt}"
                },
                {
                    "from": "gpt",
                    "value": f"{answer}"

                }
            ]
        }
        through_2020_count += 1
        json_file.append(patient_data)
    elif accession_id in all_jpgs_dict and accession_id in test_set_accn:
        line_object = {}
        line_object["question_id"] = all_jpgs_dict[accession_id]
        line_object["image"] = '/data/UCSD_cxr/jpg/' + all_jpgs_dict[accession_id]
        line_object["text"] = prompt
        line_object["category"] = ""
        jsonl_file.append(line_object)
        total_test_count += 1
    else:
        continue
print(f'---Through 2020 Completed, Found {through_2020_count} Patients---')
for j in tqdm(range(after_2020_df.shape[0])):
    accession_id = str(after_2020_df.loc[j, 'AccessionId'])
    author = 'AUTHOR: ' + after_2020_df.loc[j, 'Author']
    try:
        clinical_history = 'CLINICAL HISTORY: ' + extract_to_next_title(after_2020_df.loc[j, 'AdjustedReportText'], 'CLINICAL HISTORY')
    except:
        clinical_history = 'CLINICAL HISTORY: None'
    prompt = author + clinical_history + question
    if accession_id in all_jpgs_dict and accession_id in train_set_accn:
        try:
            findings = 'FINDINGS: ' + extract_to_next_title(after_2020_df.loc[j, 'AdjustedReportText'], 'FINDINGS')
        except:
            findings = 'FINDINGS: see IMPRESSION'
        try:
            impression = 'IMPRESSION: ' + extract_to_next_title(after_2020_df.loc[j, 'AdjustedReportText'], 'IMPRESSION:  IMPRESSION')
        except:
            continue
        answer = findings + impression
        patient_data = {
            "id": all_jpgs_dict[accession_id],
            "image": '/data/UCSD_cxr/jpg/' + all_jpgs_dict[accession_id],
            "conversations": [
                {
                    "from": "human",
                    "value": f"{prompt}"
                },
                {
                    "from": "gpt",
                    "value": f"{answer}"

                }
            ]
        }
        after_2020_count += 1
        json_file.append(patient_data)
    elif accession_id in all_jpgs_dict and accession_id in test_set_accn:
        line_object = {}
        line_object["question_id"] = all_jpgs_dict[accession_id]
        line_object["image"] = '/data/UCSD_cxr/jpg/' + all_jpgs_dict[accession_id]
        line_object["text"] = prompt
        line_object["category"] = ""
        jsonl_file.append(line_object)
        total_test_count += 1
    else:
        continue
print(f'---After 2020 Completed, Found {after_2020_count} Patients---')
assert(len(json_file) == through_2020_count + after_2020_count)
print(f'---Total Patients: {len(json_file)}---')
print(f'---Total Patients for Test: {total_test_count}---')
with open(save_json_fp, 'w') as f:
    json.dump(json_file, f, indent=4)
with open(qa_jsonl_fp, 'w') as f1:
    for record in jsonl_file:
        json_record = json.dumps(record)
        f1.write(json_record)
        f1.write('\n')
print('---Both File Dumped, All Completed---')

100%|██████████| 99493/99493 [00:01<00:00, 50658.17it/s]


---Through 2020 Completed, Found 40500 Patients---


100%|██████████| 88852/88852 [00:02<00:00, 41697.22it/s]


---After 2020 Completed, Found 56878 Patients---
---Total Patients: 97378---
---Total Patients for Test: 2247---
---Both File Dumped, All Completed---


In [24]:
new_qa_jsonl_fp = '/home/llm-hackathon/LLaVA/muchan/more_question_finding_impression.jsonl'
more_test_xray_df = pd.read_csv('/data/CheXpert-v1.0/chexpert_used_for_testing_llava.csv')
top_20_authors = list(after_2020_df['Author'].value_counts().index)[:20]
clinical_history = 'CLINICAL HISTORY: None'
more_test_jsonl_file = []
for row in range(more_test_xray_df.shape[0]):
    line_object = {}
    line_object['question_id'] = row
    line_object['image'] = '/data/' + more_test_xray_df.loc[row, 'Path']
    line_object['text'] = f'AUTHOR: {np.random.choice(top_20_authors)}, {clinical_history}. ' + question
    line_object["category"] = ""
    more_test_jsonl_file.append(line_object)
print(f'----{len(more_test_jsonl_file)} new testing images and questions prepared----')
with open(new_qa_jsonl_fp, 'w') as f2:
    for record in more_test_jsonl_file:
        json_record = json.dumps(record)
        f2.write(json_record)
        f2.write('\n')

----4999 new testing images and questions prepared----
